In [1]:
## imports

## DS & visuals
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette("pastel")
sns.set_theme(style="whitegrid")

## dattes and house functions
import datetime as dt
import eda_functions as eda

## Stats 
import scipy
from scipy import stats
## For encoding
#from sklearn.preprocessing import LabelEncoder

## Linear Regression
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse

## scikitlearn
from sklearn.model_selection import train_test_split

##
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = sns.load_dataset('penguins')
df.columns = ['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'gender']


In [3]:
df.columns

Index(['species', 'island', 'bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g', 'gender'],
      dtype='object')

In [4]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,gender
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


In [5]:
## shape and size
df.shape, df.size, df.size == df.shape[0] * df.shape[1]

((344, 7), 2408, True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   gender             333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


## EDA:

**The focus of the notebook is EDA, statistical analysis and linear regression.**

 1. Discovering and checking the overall shape, size, content. **done**
 
 2. Are any Join needed? **no**
 
 3. Validation, cleaning.**done**
 
 4. Structuring, understand trends, validate.
 
 5. **Clean, validate as you go.**
 
 6. Efective data visuals.
 
 7. Descreptive Statistics. **done**
 
     7.1 Measure of central tendency.
     
     7.2 Measures of dispersion.
     
     7.3 Measures of position.
     
     7.4 Hypothesis testing t-test&pValue. **will see**
     
 8. Check multiple linear regression assumptions or simple linear regression assumptions. 

    8.1. Linearity: Each predictor variable (Xi) is linearly related to the outcome variable (Y).

    8.2. Normality: The errors are normally distributed.*

    8.3. Independent Observations: Each observation in the dataset is independent.

    8.4. Homoscedasticity: The variance of the errors is constant or similar across the model.*

    8.5. The Data can not be multicollinear.


In [7]:
## 3. and 4.

## Observing missing data.
eda.miss_df(df)

,Total,Percent
gender,11,0.031977
bill_length_mm,2,0.005814
bill_depth_mm,2,0.005814
flipper_length_mm,2,0.005814
body_mass_g,2,0.005814
species,0,0.000000
island,0,0.000000


In [8]:
## invalid strings
eda.invalid_df(df)

,columns,nulls,invalids,unique_item
0,species,0,0,"[Adelie, Chinstrap, Gentoo]"
1,island,0,0,"[Torgersen, Biscoe, Dream]"
2,bill_length_mm,2,0,"[39.1, 39.5, 40.3, nan, 36.7, 39.3, 38.9, 39.2..."
3,bill_depth_mm,2,0,"[18.7, 17.4, 18.0, nan, 19.3, 20.6, 17.8, 19.6..."
4,flipper_length_mm,2,0,"[181.0, 186.0, 195.0, nan, 193.0, 190.0, 180.0..."
5,body_mass_g,2,0,"[3750.0, 3800.0, 3250.0, nan, 3450.0, 3650.0, ..."
6,gender,11,0,"[Male, Female, nan]"


In [9]:
## I will drop the missing values
df.dropna(inplace=True)

In [10]:
## Validating.
eda.miss_df(df)

,Total,Percent
species,0,0.0
island,0,0.0
bill_length_mm,0,0.0
bill_depth_mm,0,0.0
flipper_length_mm,0,0.0
body_mass_g,0,0.0
gender,0,0.0


In [11]:
## Validating
eda.invalid_df(df)

,columns,nulls,invalids,unique_item
0,species,0,0,"[Adelie, Chinstrap, Gentoo]"
1,island,0,0,"[Torgersen, Biscoe, Dream]"
2,bill_length_mm,0,0,"[39.1, 39.5, 40.3, 36.7, 39.3, 38.9, 39.2, 41...."
3,bill_depth_mm,0,0,"[18.7, 17.4, 18.0, 19.3, 20.6, 17.8, 19.6, 17...."
4,flipper_length_mm,0,0,"[181.0, 186.0, 195.0, 193.0, 190.0, 182.0, 191..."
5,body_mass_g,0,0,"[3750.0, 3800.0, 3250.0, 3450.0, 3650.0, 3625...."
6,gender,0,0,"[Male, Female]"


In [14]:
## 4. Structuring, understand trends, validate.

## grouping by Species and sex to observe means
df.groupby(['species', 'gender']).agg({
    'body_mass_g':'mean', 
    'flipper_length_mm':'mean', 
    'bill_depth_mm':'mean',
    'bill_length_mm':'mean',
    'species':'count',
    'body_mass_g': 'std'
})

body_mass_g  flipper_length_mm  bill_depth_mm  \
species   gender                                                  
Adelie    Female   269.380102         187.794521      17.621918   
          Male     346.811553         192.410959      19.072603   
Chinstrap Female   285.333912         191.735294      17.588235   
          Male     362.137550         199.911765      19.252941   
Gentoo    Female   281.578294         212.706897      14.237931   
          Male     313.158596         221.540984      15.718033   

                  bill_length_mm  species  
species   gender                           
Adelie    Female       37.257534       73  
          Male         40.390411       73  
Chinstrap Female       46.573529       34  
          Male         51.094118       34  
Gentoo    Female       45.563793       58  
          Male         49.473770       61

In [ ]:
## Species and sex
g = sns.catplot(
    data=df, kind='bar',
    x='species', y='body_mass_g', hue='gender',
    errorbar='sd', alpha=.6, height=6
)
g.despine(left=True)
g.set_axis_labels('', 'Body mass (g)')
g.legend.set_title('')

In [ ]:
## Population per island by species
df.groupby(['species', 'island']).agg({
    'gender':'count'})

In [ ]:
## Species and island
g = sns.catplot(
    data=df, kind='bar',
    x='species', y='body_mass_g', hue='island',
    errorbar='sd', alpha=.6, height=6
)
g.despine(left=True)
g.set_axis_labels('', 'Body mass (g)')
g.legend.set_title('')

 7. Descreptive Statistics.
 
     7.1 Measure of central tendency.
     
     7.2 Measures of dispersion.
     
     7.3 Measures of position.


In [ ]:
## 7.1, 7.2, 7.3

df.describe()

In [ ]:
## I have decided to split the data by species

Adelie = df[df['species'] == 'Adelie']
Chinstrap =  df[df['species'] == 'Chinstrap']
Gentoo =  df[df['species'] == 'Gentoo']

species = [Adelie, Chinstrap, Gentoo]

In [ ]:
eda.distribution(Adelie, 'body_mass_g')
eda.estadisticas(Adelie, 'body_mass_g')

In [ ]:
eda.empirical(Adelie, 'body_mass_g')

In [ ]:
eda.distribution(Chinstrap, 'body_mass_g')
eda.estadisticas(Chinstrap, 'body_mass_g')#,'bill_depth_mm','flipper_length_mm')

In [ ]:
eda.empirical(Chinstrap, 'body_mass_g')

In [ ]:
eda.distribution(Gentoo, 'body_mass_g')
eda.estadisticas(Gentoo, 'body_mass_g')

In [ ]:
eda.empirical(Gentoo, 'body_mass_g')

In [ ]:
## Observing relationships
sns.pairplot(df, hue='species')

### Notes

Features look normal so far, even more when divided by species.


 ...next
 
 8. Check multiple linear regression assumptions or simple linear regression assumptions. 

    8.1. Linearity: Each predictor variable (Xi) is linearly related to the outcome variable (Y). **Done**

    8.2. Normality: The errors are normally distributed.*

    8.3. Independent Observations: Each observation in the dataset is independent.

    8.4. Homoscedasticity: The variance of the errors is constant or similar across the model.*

    8.5. The Data can not be multicollinear.
    
    creating a holdout sample to better test and evaluate the results of your regression model. In order to do this more easily in Python, you must subset your x and y variables, import the `train_test_split` function from `sci-kit learn`, and then use the function. Please review the course content on holdout samples as needed before proceeding through the rest of the notebook.

In [ ]:
species[1]

In [ ]:
sns.pairplot(Adelie)

In [ ]:
sns.pairplot(Chinstrap)

In [ ]:
sns.pairplot(Gentoo)

In [ ]:
## Subset X and y variables

adelieX = Adelie[["bill_length_mm", "gender", "species"]]
adeliey = Adelie[["body_mass_g"]]

chinstrapX = Chinstrap[["bill_length_mm", "gender", "species"]]
chinstrapy = Chinstrap[["body_mass_g"]]

gentooX = Gentoo[["bill_length_mm", "gender", "species"]]
gentooy = Gentoo[["body_mass_g"]]

In [ ]:
## Create training data sets and holdout (testing) data sets
## 30% test 40% train random_seed = 42 as a pythonaite

adelieX_train, adelieX_test, adeliey_train, adeliey_test = train_test_split(adelieX, adeliey, 
                                                    test_size = 0.3, random_state = 42)
chinstrapX_train, chinstrapX_test, chinstrapy_train, chinstrapy_test = train_test_split(chinstrapX, chinstrapy, 
                                                    test_size = 0.3, random_state = 42)
gentooX_train, gentooX_test, gentooy_train, gentooy_test = train_test_split(gentooX, gentooy, 
                                                    test_size = 0.3, random_state = 42)

# Model construction

Ideally we will finish with 3 regression one per species.

## IMPORTANT NOTE
To write out the formula as a string. We write out the name of the y variable first, followed by the tilde (`~`), and then each of the X variables separated by a plus sign (`+`). **We can use `C()` to indicate a categorical variable. This will tell the `ols()` function to one hot encode those variables in the model.** 

After we've imported the `ols()` function, we can save the `ols_data` as a dataframe, create the `ols` object, fit the model, and generate summary statistics. At this point, it would make sense to double check the model assumptions about errors (homoscedasticity and normality of residuals). Please review other resources in the program as needed.

# **formula = "y ~ numFeature1 + numFeature2 + ... + C(catFeature1) + C(catFeature2)"**

#### Steps as follow:

In [ ]:
## Import ols() function from statsmodels package
## Ojo is different
from statsmodels.formula.api import ols

In [ ]:
## OSL formula as a string formula = "y ~ numFeature1 + numFeature2 + ... + C(catFeature1) + C(catFeature2)" 
adelie_ols_formula = "body_mass_g ~ bill_length_mm + C(gender) + C(species)"
chinstrap_ols_formula = "body_mass_g ~ bill_length_mm + C(gender) + C(species)"
gentoo_ols_formula = "body_mass_g ~ bill_length_mm + C(gender) + C(species)"

In [ ]:
# Create OLS dataframe adelie
ols_data_adelie = pd.concat([adelieX_train, adeliey_train], axis = 1)

# Create OLS object and fit the model adelie
OLSadelie = ols(formula = adelie_ols_formula, data = ols_data_adelie)
model_adelie = OLSadelie.fit()

# Create OLS dataframe chinstrap
ols_data_chinstrap = pd.concat([chinstrapX_train, chinstrapy_train], axis = 1)

# Create OLS object and fit the model chinstrap
OLSchinstrap = ols(formula = chinstrap_ols_formula, data = ols_data_chinstrap)
model_chinstrap = OLSchinstrap.fit()

# Create OLS dataframe gentooo
ols_data_gentoo = pd.concat([gentooX_train, gentooy_train], axis = 1)

# Create OLS object and fit the model gentooo
OLSgentoo = ols(formula = gentoo_ols_formula, data = ols_data_gentoo)
model_gentoo = OLSgentoo.fit()

## Model evaluation and interpretation

Use the `.summary()` function to get a summary table of model results and statistics.

Once we have our summary table, we can interpret and evaluate the model. In the upper half of the table, we get several summary statistics. We'll focus on `R-squared`, which tells us how much variation in body mass (g) is explained by the model. An `R-squared` of 0.85 is fairly high, and this means that 85% of the variation in body mass (g) is explained by the model.

Turning to the lower half of the table, we get the beta coefficients estimated by the model and their corresponding 95% confidence intervals and p-values. Based on the p-value column, labeled `P>|t|`, we can tell that all of the X variables are statistically significant, since the p-value is less than 0.05 for every X variable.

In [ ]:
model_adelie.summary()

In [ ]:
model_chinstrap.summary()

In [ ]:
model_gentoo.summary()

# ...and the whole data. Not split by species as before.

As a whole data set the model does better $r^2$

In [ ]:
X = df[["bill_length_mm", "gender", "species"]]
y = df[["body_mass_g"]]

# Create training data sets and holdout (testing) data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.3, random_state = 42)
# Write out OLS formula as a string
ols_formula = "body_mass_g ~ bill_length_mm + C(gender) + C(species)"

# Create OLS dataframe
ols_data = pd.concat([X_train, y_train], axis = 1)

# Create OLS object and fit the model
OLS = ols(formula = ols_formula, data = ols_data)
model = OLS.fit()

# Get model results
model.summary()

We can then interpret each of the beta coefficients for each X variable.

### C(gender) - Male
Given the name of the variable, we know that the variable was encoded as `Male = 1`, `Female = 0`. This means that female penguins are the reference point. If all other variables are constant, then we would expect a male penguin's body mass to be about 528.95 grams more than a female penguin's body mass.

### C(species) - Chinstrap and Gentoo
Given the names of these two variables, we know that Adelie penguins are the reference point. So, if we compare an Adelie penguin and a Chinstrap penguin, who have the same characteristics except their species, we would expect the Chinstrap penguin to have a body mass of about 285.39 grams less than the Adelie penguin. If we compare an Adelie penguin and a Gentoo penguin, who have the same characteristics except their species, we would expect the Gentoo penguin to have a body mass of about 1,081.62 grams more than the Adelie penguin.

### Bill length (mm)
Lastly, bill length (mm) is a continuous variable, so if we compare two penguins who have the same characteristics, except one penguin's bill is 1 millimeter longer, we would expect the penguin with the longer bill to have 35.55 grams more body mass than the penguin with the shorter bill.

# Variance Inflation Factor

The variance inflation factor is a measure for the increase of the variance of the parameter estimates if an additional variable, given by exog_idx is added to the linear regression. It is a measure for multicollinearity of the design matrix, exog.

One recommendation is that if VIF is greater than 5, then the explanatory variable given by exog_idx is highly collinear with the other explanatory variables, and the parameter estimates will have large standard errors because of this.

[WIKI](https://en.wikipedia.org/wiki/Variance_inflation_factor)

[VIF-StatsModels](https://www.statsmodels.org/v0.12.2/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

X = df[['bill_length_mm','bill_depth_mm','flipper_length_mm']]#,'body_mass_g']]

X = add_constant(X)
#print(X)
VIF = {}
VIF["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]


In [ ]:
VIF

In [ ]:
X_train.reset_index

In [ ]:
## This is nice to compare Test and train
## will show dist all nums columns.

plt.figure(figsize=(10,30))
i = 1
X_train.reset_index()
for col in X_train.columns[:-2]:
    plt.subplot(7,2,i)
    sns.histplot(x=X_train[col],color='Blue',kde=True,lw=1)
    plt.title("training data: distribution of '{}' feature".format(col));
   
    plt.subplot(7,2,i+1)
    sns.histplot(x=X_test[col],color='Red',kde=True,lw=1)
    plt.title("testing data: distribution of '{}' feature".format(col));
    i+=2
plt.tight_layout()